In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

from tqdm import tqdm

# Parameters
data_dir = './data'
batch_size = 32
num_epochs = 1
learning_rate = 0.001
num_classes = 5  # Adjust this to match the number of classes

# Check for device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transforms for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

unnormalize

# Load datasets
train_dataset = datasets.ImageFolder(data_dir, transform=transform)
val_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load a pre-trained ResNet model
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer to match our classes
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Starting training")

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        # print(labels)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation phase
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

print("Training complete.")

In [99]:
from PIL import Image

label = 4

sample_image = Image.open(f"./data/dataset_{label+1}/00000315.jpg")
sample_image = transform(sample_image)
sample_image = sample_image.unsqueeze(0)

outputs = model(sample_image)
_, predicted = torch.max(outputs, 1)
print(label, int(predicted))

4 3


In [100]:
patch_size = 30
patch = torch.rand(3, patch_size, patch_size, requires_grad=True, device=device)

patch_optimizer = optim.Adam([patch], lr=0.1)
criterion = nn.CrossEntropyLoss()

model.eval()

num_iters = 50
for i in tqdm(range(num_iters)):
    x, y = torch.randint(0, 224 - patch_size, (2,))
    patched_image = sample_image.clone()
    patched_image[0, :, x:x+patch_size, y:y+patch_size] = patch

    output = model(patched_image)

    # Compute loss
    target = torch.tensor([1], device=device)
    loss = criterion(output, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # with torch.no_grad():
    #     patch.clamp_(0, 1)


100%|██████████| 50/50 [00:08<00:00,  6.22it/s]


In [101]:

def unnormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """
    Unnormalizes a tensor by reversing the normalization applied in a Compose transform.
    Args:
        tensor (torch.Tensor): The normalized image tensor.
        mean (list): The mean used in the Normalize transform.
        std (list): The std deviation used in the Normalize transform.
    Returns:
        torch.Tensor: The unnormalized image tensor.
    """
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    return tensor * std + mean

adversarial_patch = patch.detach().cpu()
adversarial_patch_in_image = patched_image.detach().cpu()

# Make the patch POP
unnormalized_image  = unnormalize(adversarial_patch_in_image)
unnormalized_image[0, :, x:x+patch_size, y:y+patch_size] = adversarial_patch
unnormalized_image = unnormalized_image.squeeze(0).clamp(0, 1).numpy().transpose(1, 2, 0) * 255
unnormalized_image = unnormalized_image.astype('uint8')

adversarial_patch   = adversarial_patch.clamp(0, 1).numpy().transpose(1, 2, 0) * 255
adversarial_patch   = adversarial_patch.astype('uint8')

Image.fromarray(adversarial_patch).save('./patches/adversarial_patch.png')
Image.fromarray(unnormalized_image).save('./patches/adversarial_patch_in_image.png')

SyntaxError: invalid syntax (991172563.py, line 28)